# Boilerplate

There's probably a better way than what follows

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

What can we run?

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['EndpointW',
 'PacketSettingsW',
 'PacketW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'new_transport_interface']

## Transport Setup

Create a new transport interface, asynchronously. This uses the Python `asyncio` event loop, which a Jupyter Notebook already has running.

In [3]:
tiface_fut = new_transport_interface("0.0.0.0", 12346)
tiface_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f25b323f750>()]>

Await the future to get the `TransportInterface`. Note: unlike in Rust, `await` isn't necessary to drive Future completion.

In [4]:
tiface = await tiface_fut
tiface

TransportInterface{ transport: Some(<Transport>),   cmd_tx: ...,   response_rx: ...,   notify_rx: ... }

### Run the Transport in background
Here is the fun part :)

In [5]:
run_fut = tiface.run()
run_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f25b323f570>()]>

If the Future completed right away, it means something's fishy.

In [6]:
assert not run_fut.done()

Note the `transport: None` at the beginning of the following `repr` output. It is `None` because `run()` takes the `Transport`.

In [7]:
tiface

TransportInterface{ transport: None,   cmd_tx: ...,   response_rx: ...,   notify_rx: ... }

## Transport Operation

Create a `ProcessUniqueId` wrapper.

In [8]:
puid = ProcessUniqueIdW()
puid

ProcessUniqueID{<puid-0-0>}

Create another, and compare against the first.

In [9]:
puid2 = ProcessUniqueIdW()
print(repr(puid2))
assert puid != puid2

ProcessUniqueID{<puid-0-1>}


### New Remote Endpoint

Make a transport command

In [10]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
chococat_endpoint

Endpoint(157.230.134.224:2016)

In [11]:
new_endpoint_transport_cmd = TransportCmdW("newendpoint", endpoint=chococat_endpoint, timeout=100)
new_endpoint_transport_cmd

NewEndpoint { endpoint: Endpoint(157.230.134.224:2016), timeout: 100ms }

Send the command to the Transport layer and get the response from it.

In [12]:
await tiface.command_response(new_endpoint_transport_cmd)

TransportRsp::Accepted

### Send Packet to Remote

Create a `PacketSettings` wrapper. Note that the unique ID `tid` is optional, and duration is in milliseconds.

In [13]:
packet_settings = PacketSettingsW(5000) # retry after 5000ms
packet_settings

PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 250ms }

Make another transport command

Make a packet

In [14]:
getstatus_packet = PacketW(variant="getstatus", ping_nonce=98765432)
getstatus_packet

GetStatus { ping: PingPong { nonce: 98765432 } }

In [15]:


send_packets_cmd = TransportCmdW("sendpackets",
                                 endpoint=chococat_endpoint,
                                 packet_infos=[packet_settings],
                                 packets=[getstatus_packet])
send_packets_cmd

SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 250ms }], packets: [GetStatus { ping: PingPong { nonce: 98765432 } }] }

SEND IT :)

In [16]:
await tiface.command_response(send_packets_cmd)

TransportRsp::Accepted

In [26]:
notifs = tiface.get_notifications()
len(notifs)

1001

List only packets from the server

In [27]:
list(filter(lambda notif: notif.variant().lower() == 'packetdelivery', notifs))

[TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 98765432 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" } },
 TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 98765432 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" } },
 TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 98765432 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" } },
 TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 98765432 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" } },
 TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { po